In [1]:
import stanza, re, benepar, psutil, gc, json, csv, time, string, os

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [2]:
def memcheck():
    gc.collect()
    memory_info = psutil.virtual_memory()

    # Display the memory information in GB
    total_memory = memory_info.total / (1024 ** 3)
    available_memory = memory_info.available / (1024 ** 3)
    used_memory = memory_info.used / (1024 ** 3)

    print(f"Total Memory: {total_memory:.2f} GB")
    print(f"Available Memory: {available_memory:.2f} GB")
    print(f"Used Memory: {used_memory:.2f} GB")

def time_taken(start_time, end_time):
    time_taken = end_time - start_time
    hours, remainder = divmod(time_taken.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Time Taken: {hours} hours, {minutes} minutes, {seconds} seconds")

In [230]:
def display_text_in_context(text, words, window = 75):
#Finds the first occurrence of each word in target text and displays it
    
    for word in words:
        ind = text.find(word)
        if ind == -1:
            print (f"{word} not found in text")
        else:
            display_text = text[ind-window:ind+window+1]
            display_text = display_text.replace("\n", " ")
            display_text = display_text.replace("\t", " ")
            display_text = display_text.replace("  ", " ")
            print(display_text)

def display_words_in_context(text_words, words, repeat = False, window = 2):
# Finds the first occurrence of each word in target list of words. Optionally finds every subsequent occurrence     
    if not repeat:
        for word in words:
            try:
                #print("trying ", word)
                index = text_words.index(word)
                #print(f"The index of '{word}' is: {index}")
                display_text = ' '.join(text_words[index - window:index])
                display_text = display_text + " [" + word + "] "
                display_text = display_text + ' '.join(text_words[index + 1 : index + window + 1])
                print(display_text)
            except ValueError:
                print(f"'{word}' not found in the text.")
    else:
        for word in words:
            indices = [index for index, value in enumerate(text_words) if value == word]
            if len(indices) == 0:
                print(f"'{word}' not found in the text.")
            else:
                for index in indices:
                    display_text = ' '.join(text_words[index - window:index])
                    display_text = display_text + " [" + word + "] "
                    display_text = display_text + ' '.join(text_words[index + 1 : index + window + 1])
                    print(display_text)



def display_word_in_text(text, word, window=30):
    # Create regex pattern for more flexible word boundaries
    pattern = fr"(?<=[\s\-\.'\"]){re.escape(word)}(?=[\s.,!?;:'\"\-])"
    
    # Use re.finditer to get all matches and their positions
    matches = [match.start() for match in re.finditer(pattern, text)]
    
    if matches:
        matches = matches[:3]
        for match in matches:
            print(text[max(0, match - window): min(len(text), match + len(word) + window)])
        
        
    else:
        print(f"The word '{word}' was not found.")


def is_word_in_text(text, word):
    # Create regex pattern for more flexible word boundaries
    pattern = fr"(?<=[\s\-\.'\"]){re.escape(word)}(?=[\s.,!?;:'\"\-])"
    
    # Use re.finditer to get all matches and their positions
    matches = [match.start() for match in re.finditer(pattern, text)]
    return len(matches) > 0



In [42]:
def where_is(word):
    # finds if spell checker has flagged a word.  Must run spell checker first.
    if (word in spell_set_0): 
        print ("In spell set 0")
    if (word in blob_set_0):
        print ("In blob set 0")
    if (word in spell_set_0) or (word in blob_set_0):
        display_words_in_context(test_words_0, [word], False, 6)
    if (word in spell_set_1): 
        print ("In spell set 1")
    if (word in blob_set_1):
        print ("In blob set 1")
    if (word in spell_set_1) or (word in blob_set_1):
        display_words_in_context(test_words_1, [word], False, 6)
    if (word in spell_set_2): 
        print ("In spell set 2")
    if (word in blob_set_2):
        print ("In blob  set 2")
    if (word in spell_set_2) or (word in blob_set_2):
        display_words_in_context(test_words_2, [word], False, 6)
    if (word in spell_set_3): 
        print ("In spell set 3")
    if (word in blob_set_3):
        print ("In blob set 3")
    if (word in spell_set_3) or (word in blob_set_3):
        display_words_in_context(test_words_3, [word], False, 6)        
        

In [71]:
def in_which_texts(word, repeat = False):
    if word in text_1:
        print("in text_1")
        display_text_in_context(text_1, [word], window=40)
    else:
        print("not in text_1")
    
    if word in unhyphenated:
        print("in hyphenated")
        display_text_in_context(unhyphenated, [word], window=40)
    else:
        print("not in unhyphenated")
    
    if word in test_words_0:
        print ("in test_words_0")
        display_words_in_context(test_words_0, [word], repeat, 6)
    else:
        print("not in test_words_0")
    
    if word in test_words_1:
        print ("in test_words_1")
        display_words_in_context(test_words_1, [word], repeat, 6)
    else:
        print("not in test_words_1")
    
    if word in test_words_2:
        print ("in test_words_2")
        display_words_in_context(test_words_2, [word], repeat, 6)
    else:
        print("not in test_words_2")
        
    if word in test_words_3:
        print ("in test_words_3")
        display_words_in_context(test_words_3, [word], repeat, 6)
    else:
        print("not in test_words_3")
    

In [5]:
def find_hyphenated_words(text):
    # Use regex to find words that are hyphenated across lines
    # Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
    hyphenated_words = re.findall(r"(\w+)-\n(\w+)", text)

    for first_part, second_part in hyphenated_words:
        print(f"Found broken word: {first_part}-{second_part}")
    print(f"{len(hyphenated_words)} hyphenated words found, text length is: {len(text)}")
    


In [6]:
def replace_hyphenated_words(text, show_changes = False):
    # This regular expression captures words separated by a hyphen, with letters on both sides.
    pattern = r'(\b\w+)-(\w+\b)'

    # Function to handle replacement and printing
    def replacement(match):
        # Original hyphenated word
        original_word = match.group(0)
        # Replacement word (with space instead of hyphen)
        altered_word = match.group(1) + " " + match.group(2)
        
        # Print the hyphenated word that was altered
        if show_changes: print(f"Altered: {original_word} -> {altered_word}")

        return altered_word

    # Replace hyphenated words and call the replacement function
    result = re.sub(pattern, replacement, text)
    
    return result


In [7]:
def replace_hyphenated_words_alt(text, silent = True):
    # Find words that have a hyphen between them
    hyphenated_words = re.findall(r'\b\w+-\w+\b', text)
    counter = 0
    # Replace each hyphenated word with a space between words instead of the hyphen
    for word in hyphenated_words:
        new_word = word.replace('-', ' ')
        text = text.replace(word, new_word)
        counter += 1
        if not silent: print(f"Replaced '{word}' with '{new_word}'")
    print(counter)
    return text

In [8]:
def display_oddities(text):
    # Run this after creating oddities, use the text BEFORE hyphens were removed
    hyphenated_words = re.findall(r'\b\w+-\w+\b', text)
        
    for word in hyphenated_words:
        new_word = word.replace('-', ' ')
        new_word_split = new_word.split(' ')
        #print(word, new_word)
        if new_word_split[0] in oddities or new_word_split[1] in oddities:
            print (word, new_word)
            display_text_in_context(text, [word], window = 40)
            print("\n")

In [9]:
def clean_text(text, make_lower = True):
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    if make_lower:
        text = text.lower()
    # Strip any leading or trailing spaces
    text = text.strip()
    return text

# Example usage
tt = "This  is   an Example  Text with    DOUBLE  SPACES."
cleaned_text = clean_text(tt, False)
print(cleaned_text)


This is an Example Text with DOUBLE SPACES.


In [10]:
def list_to_dict(alternating_list):
    # Use dictionary comprehension with list slicing to group pairs
    return {alternating_list[i]: alternating_list[i+1]+ " " + alternating_list[i+2] for i in range(0, len(alternating_list), 3)}

# Example usage
alternating_list = ['name', 'Alice', 'bum', 'age', '30', 'years', 'city', 'New York', 'duds']
dictionary = list_to_dict(alternating_list)
print(dictionary)


{'name': 'Alice bum', 'age': '30 years', 'city': 'New York duds'}


In [11]:
# Function to find and replace specified strings in a text
def replace_strings_with_context(text, replacements, window=10, silent = False):
    """
    Replaces specified strings within a text and prints the context around each replacement.
    
    Parameters:
    - text: The original string where replacements are done.
    - replacements: A dictionary where keys are the substrings to find, and values are their replacements.
    - window: Number of characters to show before and after the replaced string for context.
    
    Returns:
    - A modified string with the replacements applied.
    """
    counter = 0
    for old, new in replacements.items():
        index = text.find(old)
        while index != -1:
            # Extract context around the found string
            start = max(index - window, 0)
            end = min(index + len(old) + window, len(text))
            before = text[start:index]
            after = text[index + len(old):end]
            before = before.replace("\n", " ")
            before = before.replace("\t", " ")
            after = after.replace("\n", " ")
            after = after.replace("\t", " ")

            # Print before, replaced, and after strings
            if not silent:
                print(f"Before: {before}[{old}]{after}")
                print(f"After:  {before}[{new}]{after}\n")

            # Replace the string in the text
            text = text[:index] + new + text[index + len(old):]
            counter += 1
            
            # Find the next occurrence of the old string
            index = text.find(old, index + len(new))

    print(f"made {counter} changes")
    return text



In [12]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"
file_path = directory_path+source_texts[1]+".txt"
print (file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day.txt


In [13]:
contractions = "Aren't	Are not Can't	Can not Couldn't	Could not Didn't	Did not Doesn't	Does not Don't	Do not Hadn't	Had not Hasn't	Has not Haven't	Have not Isn't	Is not Mightn't	Might not Mustn't	Must not Needn't	Need not Shan't	Shall not Shouldn't	Should not Wasn't was not Weren't	Were not Won't	Will not Wouldn't	Would not"
contractions = contractions + " I'd	I would You'd	You would He'd	He would She'd	She would It'd	It would We'd	We would They'd	They would Who'd	Who would There'd There would That'd That would" 
contractions = contractions + " I'll	I will You'll	You will He'll	He will She'll	She will It'll	It will We'll	We will They'll	They will Who'll	Who will That'll That will What'll What will"
contractions = contractions + " I've	I have You've	You have He's	He has She's	She has We've	We have They've	They have Who's	Who has Who've Who have"
contractions = contractions + " I'm	I am You're	You are He's	He is She's	She is It's	It is We're	We are They're	They are Who's	Who is There's There is There're There are That's That is"
contractions = clean_text(contractions, True) + " " + clean_text(contractions, False)
contractions = contractions.split(' ')
print(len(contractions))
print(contractions)
contractions = list_to_dict(contractions)
contractions["can't"] = "cannot"
contractions["Can't"] = "Cannot"

348
["aren't", 'are', 'not', "can't", 'can', 'not', "couldn't", 'could', 'not', "didn't", 'did', 'not', "doesn't", 'does', 'not', "don't", 'do', 'not', "hadn't", 'had', 'not', "hasn't", 'has', 'not', "haven't", 'have', 'not', "isn't", 'is', 'not', "mightn't", 'might', 'not', "mustn't", 'must', 'not', "needn't", 'need', 'not', "shan't", 'shall', 'not', "shouldn't", 'should', 'not', "wasn't", 'was', 'not', "weren't", 'were', 'not', "won't", 'will', 'not', "wouldn't", 'would', 'not', "i'd", 'i', 'would', "you'd", 'you', 'would', "he'd", 'he', 'would', "she'd", 'she', 'would', "it'd", 'it', 'would', "we'd", 'we', 'would', "they'd", 'they', 'would', "who'd", 'who', 'would', "there'd", 'there', 'would', "that'd", 'that', 'would', "i'll", 'i', 'will', "you'll", 'you', 'will', "he'll", 'he', 'will', "she'll", 'she', 'will', "it'll", 'it', 'will', "we'll", 'we', 'will', "they'll", 'they', 'will', "who'll", 'who', 'will', "that'll", 'that', 'will', "what'll", 'what', 'will', "i've", 'i', 'have',

In [195]:
for key, value in contractions.items():
    if " " in key or value[0] == " " or value[-1] == " ": print(key, ": ", value)
        

In [14]:
text_choice = 1


file_path = directory_path+source_texts[text_choice]+" edited_2.txt"
if os.path.isfile(file_path):
    print ("file found at ", file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
else:
    print("file not found at ", file_path)
    file_path = directory_path+source_texts[text_choice]+" edited_1.txt"
    if os.path.isfile(file_path):
        print ("file found at ", file_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    else:
        print("file not found at ", file_path)
        file_path = directory_path+source_texts[text_choice]+" edited.txt"
        if os.path.isfile(file_path):
            print ("file found at ", file_path)
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
        else:
            print("file not found at ", file_path)


file not found at  C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day edited_2.txt
file found at  C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day edited_1.txt


In [15]:
print (len(text))
text_1 = replace_strings_with_context(text, contractions, window=10, silent = True)
print(len(text_1))

423406
made 657 changes
424295


In [16]:
unhyphenated_alt = replace_hyphenated_words_alt(text_1)

176


In [17]:
test_words_0 = text_1.translate(str.maketrans('', '', string.punctuation)).split()
test_words_1 = text_1.translate(str.maketrans('', '', string.punctuation)).lower().split()
unhyphenated = replace_hyphenated_words(text_1)
test_words_2 = unhyphenated.translate(str.maketrans('', '', string.punctuation)).split()
test_words_3 = unhyphenated.translate(str.maketrans('', '', string.punctuation)).lower().split()

In [18]:
start_time = datetime.now()
print(f"Starting processing at: {start_time.strftime('%H:%M:%S')}")

# Find invalid words
blob_set_0 = set()
for word in test_words_0:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_0.add(word)

blob_set_1 = set()
for word in test_words_1:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_1.add(word)

blob_set_2 = set()
for word in test_words_2:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_2.add(word)

blob_set_3 = set()
for word in test_words_3:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_3.add(word)
        

end_time = datetime.now()
print(f"Completed processing at: {end_time.strftime('%H:%M:%S')}")
time_taken(start_time, end_time)

print(len(blob_set_0), len(blob_set_1), len(blob_set_2), len(blob_set_3))

Starting processing at: 08:43:25
Completed processing at: 09:02:23
Time Taken: 0 hours, 18 minutes, 57 seconds
1244 971 1144 871


In [19]:
print(f"errors found in base text: {len(blob_set_0)}")
print(f"errors found with case lowered: {len(blob_set_1)}")
print(f"errors found with hyphens removed:  {len(blob_set_2)}")     
print(f"errors found with hyphens removed and case lowered: {len(blob_set_3)}")     
print(f"errors eliminated by removing hyphens:  {len(blob_set_0 - blob_set_2)} or in lower case: {len(blob_set_1 - blob_set_3)}")
print(f"errors introduced by removing hyphens:  {len(blob_set_2 - blob_set_0)} or in lower case: {len(blob_set_3 - blob_set_1)}")
print(f"errors eliminated by lowering case:  {len(blob_set_0 - blob_set_1)} or without hyphens: {len(blob_set_2 - blob_set_3)}")
print(f"errors introduced by lowering case:  {len(blob_set_1 - blob_set_0)} or without hyphens: {len(blob_set_3 - blob_set_2)}")

errors found in base text: 1244
errors found with case lowered: 971
errors found with hyphens removed:  1144
errors found with hyphens removed and case lowered: 871
errors eliminated by removing hyphens:  124 or in lower case: 123
errors introduced by removing hyphens:  24 or in lower case: 23
errors eliminated by lowering case:  428 or without hyphens: 424
errors introduced by lowering case:  155 or without hyphens: 151


In [20]:
oddities = sorted(list(blob_set_2-blob_set_0))
print(' '.join(oddities))
#display_words_in_context(test_words_2, oddities, window = 5)


Heinz Karl Semitic bores chock hog humouredness inwaiting jacketed longue loverStevens martialling matching mating newt ofhand pep sheeted sheetings slated sleights tiles ups vident


In [22]:
oddities_lower = sorted(list(blob_set_3-blob_set_1))
print(' '.join(oddities_lower))

bores chock heinz hog humouredness inwaiting jacketed longue loverstevens martialling matching mating newt ofhand pep semitic sheeted sheetings slated sleights tiles ups vident


In [23]:
missed = [word for word in oddities_lower if word not in blob_set_3]
print(' '.join(missed))

In [25]:
display_oddities(text_1)

chaise-longue chaise longue
 down on a table, seated himself on the chaise-longue, and stretched out his legs


dust-sheeted dust sheeted
orridor up on the second floor would be dust-sheeted, leaving all the main ground


good-humouredness good humouredness
 was participating in some way with the good-humouredness with which he was carry


newt-mating newt mating
e-ranging topics such as  falconing or newt-mating - attributes none of which my


twelve-bores twelve bores
ining room. Perhaps you will permit the twelve-bores to be used?"  And according


court-martialling court martialling
ad been calls for the removal, even the court-martialling, of the general concern


self-vident self vident
ssing him as 'William' should have been self-vident to you." "Mr Stevens, I may 


roof-tiles roof tiles
o view from his small window other than roof-tiles and guttering. The oil lamp be


Karl-Heinz Karl Heinz
treaty, but by his friendship with Herr Karl-Heinz Bremann. Herr Bremann first v


pep-

In [26]:
# Use spellchecker to validate words
# Initialize spell checker
spell = SpellChecker()


# Find words not in the dictionary
spell_set_0 = {word for word in test_words_0 if word not in spell}
spell_set_1 = {word for word in test_words_1 if word not in spell}
spell_set_2 = {word for word in test_words_2 if word not in spell}
spell_set_3 = {word for word in test_words_3 if word not in spell}

print(len(spell_set_0), len(spell_set_1), len(spell_set_2), len(spell_set_3))


452 447 366 361


In [27]:
both_3 = spell_set_3 & blob_set_3

In [28]:
print(len(both_3))

234


In [48]:
where_is("witha")

In spell set 0
In blob set 0
that Lord Darlington was liaising covertly [witha] known enemy is just conveniently forgetting
In spell set 1
In blob set 1
that lord darlington was liaising covertly [witha] known enemy is just conveniently forgetting


In [63]:
where_is("with-a")

In [64]:
where_is("self-vident")

In [65]:
where_is("not-seek")

In [66]:
where_is("be-that")

In [72]:
in_which_texts("with-a")

in text_1
t Lord Darlington was liaising covertly with-a known enemy is just conveniently f
not in unhyphenated
not in test_words_0
not in test_words_1
not in test_words_2
not in test_words_3


In [73]:
in_which_texts("self-vident")

in text_1
ssing him as 'William' should have been self-vident to you." "Mr Stevens, I may 
not in unhyphenated
not in test_words_0
not in test_words_1
not in test_words_2
not in test_words_3


In [74]:
in_which_texts("not-seek")

in text_1
rs is no reason in itself why we should not-seek a more convenient arrangement fr
not in unhyphenated
not in test_words_0
not in test_words_1
not in test_words_2
not in test_words_3


In [75]:
in_which_texts("be-that")

in text_1
ou Englishmen", Mr Lewis said, seems to be-that you do not really hate the German
not in unhyphenated
not in test_words_0
not in test_words_1
not in test_words_2
not in test_words_3


In [49]:
where_is("selfvident")

In spell set 0
In blob set 0
him as William should have been [selfvident] to you Mr Stevens I may
In spell set 1
In blob set 1
him as william should have been [selfvident] to you mr stevens i may


In [50]:
where_is("notseek")

In spell set 0
In blob set 0
reason in itself why we should [notseek] a more convenient arrangement from here
In spell set 1
In blob set 1
reason in itself why we should [notseek] a more convenient arrangement from here


In [51]:
where_is("bethat")

In spell set 0
In blob set 0
Englishmen Mr Lewis said seems to [bethat] you do not really hate the
In spell set 1
In blob set 1
englishmen mr lewis said seems to [bethat] you do not really hate the


In [52]:
where_is("carryon")

In blob set 0
father would have wished me to [carryon] just now Of course Mr Stevens
In blob set 1
father would have wished me to [carryon] just now of course mr stevens
In blob  set 2
father would have wished me to [carryon] just now Of course Mr Stevens
In blob set 3
father would have wished me to [carryon] just now of course mr stevens


In [76]:
in_which_texts("witha")

not in text_1
not in unhyphenated
in test_words_0
that Lord Darlington was liaising covertly [witha] known enemy is just conveniently forgetting
in test_words_1
that lord darlington was liaising covertly [witha] known enemy is just conveniently forgetting
not in test_words_2
not in test_words_3


In [54]:
where_is("ell")

In blob set 0
matter we have been debating T [ell] me do you suppose the debt
In blob set 1
matter we have been debating t [ell] me do you suppose the debt
In blob  set 2
matter we have been debating T [ell] me do you suppose the debt
In blob set 3
matter we have been debating t [ell] me do you suppose the debt


In [61]:
in_which_texts("with-a")

in text_1
t Lord Darlington was liaising covertly with-a known enemy is just conveniently f
not in unhyphenated


In [81]:
where_is("T")

In [82]:

print(' '.join(word for word in blob_set_3))
# display_words_in_context(test_words_2, spell_only_2 - spell_only_1, False, 3)

reassurance failings comprised debased anysuch remiss jeopardize banqueting posit carolyn dismissing teas unorthodox hmm imputing motivated idealistic teapot tragically bolshevik throwaway diners badman obsessions facets heretical compton mantelshelf alerted swings confusions assignations driveway kernel benns uncertainly dorset saltash furthering gesturing crosby switching grounding matronly bemused undeniable helston roas oversee analysed contemplating thawed unimaginable yesterdays deterioration nonsensical duplicitous replacements overseeing ladys reversing exeter rapping crudeness reciprocate titled replenish eulogized unnerving trolleys valeting unforgettable ungrammatical advertises irredeemable distinguishedand poplars greyed adjoin begs frogmarched speculations al1 wallpaper bulrushes engagingly needlework investigating implications enthuse reginalds cue morphy canyons reiterate complicit practising inoffensive artistes ostentatious longue herman endearing inwaiting unrealisti

In [136]:
#copy and extract the ones that could be real errors, paste them back here
errors_raw = "anysuch posit badman roas distinguishedand al1 inwaiting avery ell allhis farradayscircle mauvis disembarkation verypleased ked ofthe loverstevens 0f itinvolved overath startseeing iarrived israther seehe cansee beyondsuch carryon neighbourss thevery gotto civvy wakeling wer canbe prise isall ill1terest ting laval docrucially owardst ifi infact giffen het chainbers tosit nottake andthe thelikes ot mursden pres canonly allshot amvery simplyaccepting ohnothing dh friendsand lto lookingback congenia1 vident terming ican lotyou istrue tj1e putout agre confidentia1 specia1 arec loc contemplatin8"
errors_raw = errors_raw.split()
errors_raw = sorted(errors_raw)
print(len(errors_raw))
print (' '.join(errors_raw))

76
0f agre al1 allhis allshot amvery andthe anysuch arec avery badman beyondsuch canbe canonly cansee carryon chainbers civvy confidentia1 congenia1 contemplatin8 dh disembarkation distinguishedand docrucially ell farradayscircle friendsand giffen gotto het iarrived ican ifi ill1terest infact inwaiting isall israther istrue itinvolved ked laval loc lookingback lotyou loverstevens lto mauvis mursden neighbourss nottake ofthe ohnothing ot overath owardst posit pres prise putout roas seehe simplyaccepting specia1 startseeing terming thelikes thevery ting tj1e tosit verypleased vident wakeling wer


In [177]:
errors_found = (word for word in errors_raw if is_word_in_text(text_1, word))
errors_found = sorted(errors_found)
print(len(errors_found))
errors_not_found = [word for word in errors_raw if word not in errors_found]
print(len(errors_not_found))

53
23


In [91]:
display_words_in_context(test_words_3, errors_not_found)

house in [allshot] oxfordshire clearly
sir james [chainbers] a good
relieved way [dh] mr stevens
rich as [distinguishedand] in my
now mr [farradayscircle] naturally being
firm of [giffen] and co
was that [iarrived] here at
agreement but [ican] say for
forgive me [ifi] were to
this cottage [infact] as i
of ladies [inwaiting] and footmen
he put [itinvolved] in overseeing
was m [laval] really intending
rather a [lotyou] see it
a nature [loverstevens] he said
essential then [lto] keep ones
sir reginald [mauvis] remarking that
village of [mursden] perhaps mursden
thirties mr [neighbourss] name seemed
mr stevens [ohnothing] in particular
stevens baron [overath] old friend
from being [putout] i felt
up at [tj1e] end of
stature lord [wakeling] lord camberley


In [178]:
print(' '.join(errors_not_found))

allshot chainbers dh distinguishedand farradayscircle giffen iarrived ican ifi infact inwaiting itinvolved laval lotyou loverstevens mauvis mursden neighbourss ohnothing overath tj1e wakeling wer


In [92]:
print(' '.join(errors_not_found))

allshot chainbers dh distinguishedand farradayscircle giffen iarrived ican ifi infact inwaiting itinvolved laval lotyou loverstevens lto mauvis mursden neighbourss ohnothing overath putout tj1e wakeling


In [180]:
suggestions = "Allshot Chainbers dh distinguished,and Farraday'scircle Giffen Iarrived Ican ifI in-fact in-waiting it,involved Laval lot,you lover,Stevens Mauvis Mursden Neighbours's oh,nothing Overath put,out Tj1e Wakeling"
suggestions = suggestions.split(' ')
suggestions_found = (word for word in suggestions if is_word_in_text(text_1, word))
suggestions_found = sorted(suggestions_found)
print(len(suggestions_found))
suggestions_not_found = [word for word in suggestions if word not in suggestions_found]
print(len(suggestions_not_found))

16
7


In [181]:
for error, suggestion in zip(errors_not_found, suggestions):
    if suggestion not in suggestions_found:
        display_words_in_context(test_words_3, [error], window = 14)

to raise the very topic for she said in something of a relieved way [dh] mr stevens it is just someone i knew once when i was at granchester
the society did not regard the houses of businessmen or the newly rich as [distinguishedand] in my opinion this piece of out dated thinking crucially undermined any serious authority
what else you could do at this hour the wife would never forgive me [ifi] were to let you away into the night thus it was that i came
the whole village was aware of my mishap and subsequent arrival at this cottage [infact] as i was soon to discover this was very close to being the case
little reason to be oh what things in particular do you mean mr stevens [ohnothing] in particular mrs benn oh mr stevens you really must tell me well for
been initially so preoccupied with the peace treaty when it was drawn up at [tj1e] end of the great war and i think it is fair to say that
mr marshall and mr lane have served only gentlemen of indisputable moral stature lord [wakeling]

For the ones that can't be found, search for some of the text window, making sure it's a bit that won't have capitals or punctuation

In [182]:
search = list()
search.append("said in something of a relieved way")
search.append("did not regard the houses of businessmen")
search.append("let you away into the night")
search.append("subsequent arrival at this cottage")
search.append("little reason to be")
search.append("indebted for the hospitality")
search.append("peace treaty when it was drawn up")
for snippet in search: print(snippet)

said in something of a relieved way
did not regard the houses of businessmen
let you away into the night
subsequent arrival at this cottage
little reason to be
indebted for the hospitality
peace treaty when it was drawn up


In [186]:
for snippet in search:
    display_text_in_context(text_1, [snippet], window = 100)
    

 it was almost as though she had been long awaiting an opportunity to raise the very topic. For she said in something of a relieved way: "Dh, Mr Stevens, it is just someone I knew once when I was at G
f is far from sufficient to satisfy requirements.' It was made clear, furthermore, that the Society did not regard the houses of businessmen or the 'newly rich' as 'distinguished',and in my opinion thi
r, I do not know what else you could do at this hour. The wife would never forgive me if'-I were to let you away into the night." Thus it was that I came to accept the kind hospitality of Mr and Mrs T
ld stop by." The way he said this seemed to suggest the whole village was aware of my 'mishap' and subsequent arrival at this cottage. Infact, as I was soon to discover, this was very close to being t
rtain things you wrote in your letter. I was a little worried when I read them, but I see now I had little reason to be." "Oh? What things in particular do you mean, Mr Stevens?" "Oh,nothing in p

Now redo the suggestions:

In [187]:
print(suggestions_not_found)
#paste below

['dh', 'distinguished,and', 'ifI', 'in-fact', 'oh,nothing', 'put,out', 'Tj1e']


In [188]:
new_suggestions = ['Dh', "'distinguished',and", "if'-I", 'Infact', 'Oh,nothing', "'putout'", 'tJ1e']

In [189]:
for suggestion in new_suggestions:
    if suggestion not in text_1:
        print(suggestion, " still not found")

In [190]:
errors = errors_found[:]
print(len(errors_found), len(errors))


errors.extend(suggestions_found)

print(len(errors_found), len(errors))
errors.extend(new_suggestions)

print(len(errors_raw), len(errors))
# These two should be the same

53 53
53 69
76 76


In [149]:
print(errors)

['0f', 'agre', 'al1', 'allhis', 'amvery', 'andthe', 'anysuch', 'arec', 'avery', 'badman', 'beyondsuch', 'canbe', 'canonly', 'cansee', 'carryon', 'civvy', 'confidentia1', 'congenia1', 'contemplatin8', 'disembarkation', 'docrucially', 'ell', 'friendsand', 'gotto', 'het', 'ill1terest', 'isall', 'israther', 'istrue', 'ked', 'loc', 'lookingback', 'nottake', 'ofthe', 'ot', 'owardst', 'posit', 'pres', 'prise', 'roas', 'seehe', 'simplyaccepting', 'specia1', 'startseeing', 'terming', 'thelikes', 'thevery', 'ting', 'tosit', 'verypleased', 'vident', 'wer', 'Allshot', 'Chainbers', "Farraday'scircle", 'Giffen', 'Iarrived', 'Ican', 'Laval', 'Mauvis', 'Mursden', "Neighbours's", 'Overath', 'Wakeling', 'in-waiting', 'it,involved', 'lot,you', 'lover,Stevens', 'Dh', "'distinguished',and", "if'-I", 'Infact', '.lto', 'Oh,nothing', "'putout'", 'tJ1e']


In [198]:
#paste the output below to make the corrections dictionary using the display as reference. If the word is ok, just delete the line instead instead of making a correction
#We will do this in chunks to save having to do lots of scrolling

def split_up(lst, chunk_size=10):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

In [199]:
corrections = dict()

In [200]:
chunks = split_up(errors)
print("number of chunks = ", len(chunks), " (including 0)")

number of chunks =  8  (including 0)


In [214]:
chunk = 0
for error in chunks[chunk]:
    print('corrections["'+error+'"] = "'+error+'"')
print("\n")
for error in chunks[chunk]:
    find_word_in_text(text_1, error, window=70)

corrections["0f"] = "0f"
corrections["agre"] = "agre"
corrections["al1"] = "al1"
corrections["allhis"] = "allhis"
corrections["amvery"] = "amvery"
corrections["andthe"] = "andthe"
corrections["anysuch"] = "anysuch"
corrections["arec"] = "arec"
corrections["avery"] = "avery"
corrections["badman"] = "badman"


 days when you could act put of your noble instincts are over. Except 0f course, you here in Europe do not yet seem to know it. Gentlemen like
buting only statements such as, 'Of course, Mr Stevens,' or, 'I quite agre, Mr Stevens,' I finally said to her:


"I am sorry, Miss Kenton, but 
 outlining just what this gentleman has been saying to me - about you al1. And with a most clumsy technique, the audacity and crudeness of whic
oubt that a desire to see 'justice in this world' lay at the heart of allhis actions.

It was not long after that evening there came the sad news 
 Have a go, mate."

"Oh dear, no, thank you, it is quite all right. I amvery sorry, I am afraid the travelling 

In [215]:
corrections["0f"] = "of"
corrections["agre,"] = "agree,"
corrections["al1"] = "all"
corrections["allhis"] = "all his"
corrections["amvery"] = "am very"
corrections["andthe"] = "and the"
corrections["anysuch"] = "any such"
corrections["arec"] = "care"
corrections["avery"] = "a very"
corrections["badman"] = "bad man"

In [203]:
chunk = 1
for error in chunks[chunk]:
    print('corrections["'+error+'"] = "'+error+'"')
print("\n")
for error in chunks[chunk]:
    find_word_in_text(text_1, error, window=70)

corrections["beyondsuch"] = "beyondsuch"
corrections["canbe"] = "canbe"
corrections["canonly"] = "canonly"
corrections["cansee"] = "cansee"
corrections["carryon"] = "carryon"
corrections["civvy"] = "civvy"
corrections["confidentia1"] = "confidentia1"
corrections["congenia1"] = "congenia1"
corrections["contemplatin8"] = "contemplatin8"
corrections["disembarkation"] = "disembarkation"


s suit and his shirt and run about screaming. In a word, 'dignity' is beyondsuch persons. We English have an important advantage over foreigners in th
arry Smith that one's 'dignity' is conditional on being able to do so canbe seen for the nonsense it is. Let us establish this quite clearly: a b
nguished household is a prerequisite of 'greatness'. A 'great' butler canonly be, surely, one who can point to his years of service and say that he
ys, it is too misty and it is like it is vanished altogether. But you cansee for yourself, on a fine day like this, it is a nice sight. "

"Deligh
ust at this moment. 

In [204]:
corrections["beyondsuch"] = "beyond such"
corrections["canbe"] = "can be"
corrections["canonly"] = "can only"
corrections["cansee"] = "can see"
corrections["carryon"] = "carry on"
corrections["civvy"] = "civvy"
corrections["confidentia1"] = "confidential"
corrections["congenia1"] = "congenial"
corrections["contemplatin8"] = "contemplating"
corrections["disembarkation"] = "disembarkation"

In [205]:
chunk = 2
for error in chunks[chunk]:
    print('corrections["'+error+'"] = "'+error+'"')
print("\n")
for error in chunks[chunk]:
    find_word_in_text(text_1, error, window=70)

corrections["docrucially"] = "docrucially"
corrections["ell"] = "ell"
corrections["friendsand"] = "friendsand"
corrections["gotto"] = "gotto"
corrections["het"] = "het"
corrections["ill1terest"] = "ill1terest"
corrections["isall"] = "isall"
corrections["israther"] = "israther"
corrections["istrue"] = "istrue"
corrections["ked"] = "ked"


f what true 'dignity' is. And let me now posit this: 'dignity' has to docrucially with a butler's ability not to abandon the professional being he inha
 you.

We need your help on a certain matter we have been debating. T ell me, do you suppose the debt situation regarding America is a signific
you are referring to."

"You do not understand, Stevens. Well, we are friendsand so I will put it to you frankly. Over the last few years, his lordshi
ll as you used to. But it is the same for all of us, see? We have all gotto put our feet up at some point. Look at me. Been happy as a lark since
ll you the truth, we do not miss it so much. There is a few houses 

In [206]:
corrections["docrucially"] = "do crucially"
corrections["T ell"] = "Tell"
corrections["friendsand"] = "friends and"
corrections["gotto"] = "got to"
corrections["het"] = "the"
corrections["ill1terest"] = "interest"
corrections["isall"] = "is all"
corrections["israther"] = "is rather"
corrections["istrue"] = "is true"
corrections["loc ked"] = "locked"

In [207]:
chunk = 3
for error in chunks[chunk]:
    print('corrections["'+error+'"] = "'+error+'"')
print("\n")
for error in chunks[chunk]:
    find_word_in_text(text_1, error, window=70)

corrections["loc"] = "loc"
corrections["lookingback"] = "lookingback"
corrections["lto"] = "lto"
corrections["nottake"] = "nottake"
corrections["ofthe"] = "ofthe"
corrections["ot"] = "ot"
corrections["owardst"] = "owardst"
corrections["posit"] = "posit"
corrections["pres"] = "pres"
corrections["prise"] = "prise"


t me, that he said:

"You realize, Stevens, I do not expect you to be loc ked up here in this house all the time I am away. Why do not you take
was, in the truest terms, 'attached to a distinguished household'. In lookingback over my career thus far, my chief satisfaction derives from what I ac
w all that is best about service in England. It is essential, then,

.lto keep one's attention focused on the present; to guard against any com
nd rather rough.

"I am telling you, sir, you will be sorry if you do nottake a walk up there. And you never know. A couple more years and it might
m - a venue chosen to accommodate the 'off the record' nature of many ofthe attendances. In fact

In [209]:
corrections["lookingback"] = "looking back"
corrections[".lto"] = "to"
corrections["nottake"] = "not take"
corrections["ofthe"] = "of the"
corrections["ot"] = "to"
corrections["owardst"] = "towards"
corrections["posit"] = "posit"
corrections["pres en t"] = "present"
corrections["prise"] = "prise"


In [208]:
display_text_in_context(text_1, [".lto"])

to show all that is best about service in England. It is essential, then, .lto keep one's attention focused on the present; to guard against any compl


In [210]:
chunk = 4
for error in chunks[chunk]:
    print('corrections["'+error+'"] = "'+error+'"')
print("\n")
for error in chunks[chunk]:
    find_word_in_text(text_1, error, window=70)

corrections["putout"] = "putout"
corrections["roas"] = "roas"
corrections["seehe"] = "seehe"
corrections["simplyaccepting"] = "simplyaccepting"
corrections["specia1"] = "specia1"
corrections["startseeing"] = "startseeing"
corrections["terming"] = "terming"
corrections["thelikes"] = "thelikes"
corrections["thevery"] = "thevery"
corrections["ting"] = "ting"


e termed it. In any case, I replied with a smile that far from being 'putout', I felt extremely indebted for the hospitality I was receiving. By t
Mortimer - or else her apron - the room was dominated by the smell of roas ting.

Dr Meredith rose and said: "My condolences, Stevens. He suffer
d the grounds. He was clutching his attaché case as usual and I could seehe was strolling slowly along the path that runs the outer perimeter of 
y intelligently bestowed. What is there 'undignified' in this? One is simplyaccepting an inescapable truth: that the likes of you and I will never be in a 
d. Oh no, that would not do at all."

"Oh, dea

In [211]:
corrections["putout"] = "put out"
corrections["roas ting"] = "roasting"
corrections["seehe"] = "see he"
corrections["simplyaccepting"] = "simply accepting"
corrections["specia1"] = "special"
corrections["startseeing"] = "start seeing"
corrections["terming"] = "terming"
corrections["thelikes"] = "the likes"
corrections["thevery"] = "the very"
corrections["ting"] = "ting"

In [212]:
chunk = 5
for error in chunks[chunk]:
    print('corrections["'+error+'"] = "'+error+'"')
print("\n")
for error in chunks[chunk]:
    find_word_in_text(text_1, error, window=70)

corrections["tosit"] = "tosit"
corrections["verypleased"] = "verypleased"
corrections["vident"] = "vident"
corrections["Allshot"] = "Allshot"
corrections["Chainbers"] = "Chainbers"
corrections["Farraday'scircle"] = "Farraday'scircle"
corrections["Giffen"] = "Giffen"
corrections["Iarrived"] = "Iarrived"
corrections["Ican"] = "Ican"
corrections["Laval"] = "Laval"


 the lights would be switched on 'fairly soon', and so I have decided tosit down here on this bench and await the event. I have a good view from 
worth speaking of."

"I am not disappointed at all, Miss Kenton. I am verypleased for you and for all of us. I will admit, you have had some modest suc
ur age and standing addressing him as 'William' should have been self-vident to you."

"Mr Stevens, I may not have been a housekeeper for long, bu
tler - to a Mr and Mrs Muggeridge in their relatively modest house in Allshot, Oxfordshire. Clearly the story meant much to him. My father's genera
 for I remember mentioning it to Mr Graha

In [213]:
corrections["tosit"] = "to sit"
corrections["verypleased"] = "very pleased"
corrections["self-vident"] = "sel-evident"
corrections["Allshot"] = "Allshot"
corrections["Chainbers"] = "Chambers"
corrections["Farraday'scircle"] = "Farraday's circle"
corrections["Giffen"] = "Giffen"
corrections["Iarrived"] = "I arrived"
corrections["Ican"] = "I can"
corrections["Laval"] = "Laval"
    

In [216]:
chunk = 6
for error in chunks[chunk]:
    print('corrections["'+error+'"] = "'+error+'"')
print("\n")
for error in chunks[chunk]:
    find_word_in_text(text_1, error, window=70)

corrections["Mauvis"] = "Mauvis"
corrections["Mursden"] = "Mursden"
corrections["Neighbours's"] = "Neighbours's"
corrections["Overath"] = "Overath"
corrections["Wakeling"] = "Wakeling"
corrections["in-waiting"] = "in-waiting"
corrections["it,involved"] = "it,involved"
corrections["lot,you"] = "lot,you"
corrections["lover,Stevens"] = "lover,Stevens"
corrections["Dh"] = "Dh"


ears ago, Mr Rayne, who travelled to America as valet to Sir Reginald Mauvis, remarking that a taxi driver in New York regularly addressed his far
eral nearby destinations. One of these destinations is the village of Mursden. Perhaps 'Mursden' will ring a bell for you, as it did for me upon my
ations. One of these destinations is the village of Mursden. Perhaps 'Mursden' will ring a bell for you, as it did for me upon my first spotting it
o make a slight detour from my planned route just to see the village. Mursden, Somerset, was where the firm of Giffen and Co. was once situated, an
is was a typical case. For two 

In [217]:
corrections["Mauvis"] = "Mauvis"
corrections["Mursden"] = "Mursden"
corrections["Neighbours's"] = "Neighbours's"
corrections["Overath"] = "Overath"
corrections["Wakeling"] = "Wakeling"
corrections["in-waiting"] = "in-waiting"
corrections["it,involved"] = "it, involved"
corrections["lot,you"] = "lot, you"
corrections["lover,Stevens"] = "lover, Stevens"
corrections["Dh"] = "Oh"

In [218]:
chunk = 7
for error in chunks[chunk]:
    print('corrections["'+error+'"] = "'+error+'"')
print("\n")
for error in chunks[chunk]:
    find_word_in_text(text_1, error, window=70)

corrections["'distinguished',and"] = "'distinguished',and"
corrections["if'-I"] = "if'-I"
corrections["Infact"] = "Infact"
corrections["Oh,nothing"] = "Oh,nothing"
corrections["'putout'"] = "'putout'"
corrections["tJ1e"] = "tJ1e"


ciety did not regard the houses of businessmen or the 'newly rich' as 'distinguished',and in my opinion this piece of out-dated thinking crucially undermined a
 what else you could do at this hour. The wife would never forgive me if'-I were to let you away into the night."

Thus it was that I came to acc
lage was aware of my 'mishap' and subsequent arrival at this cottage. Infact, as I was soon to discover, this was very close to being the case; I 
n to be."

"Oh? What things in particular do you mean, Mr Stevens?"

"Oh,nothing in particular, Mrs Benn."

"Oh, Mr Stevens, you really must tell me."
he termed it. In any case, I replied with a smile that far from being 'putout', I felt extremely indebted for the hospitality I was receiving. By th
nitially so pre

In [219]:
corrections["'distinguished',and"] = "'distinguished', and"
corrections["if'-I"] = "if I"
corrections["Infact"] = "In fact"
corrections["Oh,nothing"] = "Oh, nothing"
corrections["'putout'"] = "'put out'"
corrections["tJ1e"] = "the"


In [227]:
# Add any others that were spotted but not yet included
corrections["notseek"] = "not seek"
corrections["witha"] = "with a"
corrections["bethat"] = "be that"

In [229]:
# check for potentially erroneous overwrites
for key in corrections:
    occurrences = is_word_in_text(text_1, key)#print(key)

IndentationError: expected an indented block (1354065872.py, line 3)

In [33]:
for word in errors_found:
    print(word)
    display_text_in_context(text_1, [word], window = 40)

5s
ons and none of us, not even the Senior 5s, dared make a sound. There was a real
becausethey
ble about us having sex with each other becausethey would then want to have sex w
betweenthem
 The idea the guardians had differences betweenthem, that never occurred to us." 
bric
ut, he put the mags down on top of some bricks stacked outside the boiler hut-som
canunzip
 it is right on the elbow like that, it canunzip. All you have to do is bend your
carefulwho
y and began telling us how we had to be carefulwho we had sex with. Not just beca
coupleshinting
of snogging and touching up, maybe; and coupleshinting they were having proper se
definitelyhad
 that two of the girls I was closest to definitelyhad done it. Laura with Rob D.,
doi
o completing and so that is what he was doing: getting me to describe things to h
fromme
you do not do it that way! Just take it fromme, you do not do it that way! You de
howshe
ago. "None of us stopped to think about howshe felt , Miss Lucy herself. We never


In [ ]:
print(errors_not_found_2)

In [ ]:

suggestions = ['alost', 'coupleand', 'cwho', 'itbut', 'Iwould', 'Jeansruth', 'jwho', 'Keffers', 'Kefferss', 'kshed', 'Lauras', 'LPs', 'Metchley', 'Portway', 'shapei', 'virginmeaning', 'Wainright', 'Walkman', 'Walkmans', 'wavecrest']

In [ ]:
print(len(suggestions))
suggestions_found = (word for word in suggestions if word in text)
suggestions_found = sorted(suggestions_found)
print(len(suggestions_found))
suggestions_not_found = [word for word in suggestions if word not in suggestions_found]
print(len(suggestions_not_found))

In [ ]:
print(suggestions_found)

In [ ]:
errors = ['0f', 'agre', 'al1', 'allhis', 'andthe', 'anysuch', 'arec', 'avery', 'be-that', 'beyondsuch', 'canbe', 'canonly', 'cansee', 'carryon', 'Chainbers', 'confidentia1', 'contemplatin8', 'Dh', "'distinguished',and", 'docrucially', "don'ttake", 'T ell', "Farraday'scircle", 'friendsand', 'gotto', 'het', 'Iarrived', 'Ican', "if'-I", 'ill1terest', "I'mvery", 'Infact', 'istrue', 'it,involved', "it'srather", 'loc ked', 'lookingback', 'lot,you', 'lover,Stevens', 'lto', 'not-seek', 'ofthe', 'Oh,nothing', 'ot', 'owardst', 'putout', 'roas ting', 'seehe', 'self-vident', 'simplyaccepting', 'startseeing', "that'sall", 'thelikes', 'thevery', 'tJ1e', 'tosit', 'verypleased', 'village,sir', 'wer', 'with-a']


In [ ]:
print(len(errors))

In [ ]:
print("distinguishedand" in errors_raw)
print("distinguishedand" in errors)

In [ ]:
errors_tt == errors_raw

In [ ]:
corrections = ['of', 'agree', 'all', 'all his', 'and the', 'any such', 'care', 'a very', 'be that', 'beyond such', 'can be', 'can only', 'can see', 'carry on', 'Chambers', 'confidential', 'contemplating', 'Oh', "'distinguished', and", 'do crucially', "don't take", 'Tell', 'Farradays Circle', 'friends and', 'got to', 'the', 'I arrived', 'I can', 'if I', 'interest', "I'm very", 'In fact', 'is true', 'it involved', "its rather", 'locked', 'looking back', 'lot, you', 'lover, Stevens', 'to', 'not seek', 'of the', 'Oh, nothing', 'to', 'towards', 'put out', 'roasting', 'see he', 'self-evident', 'simply accepting', 'start seeing', "that's all", 'the likes', 'the very', 'the', 'to sit', 'very pleased', 'village, sir', 'were', 'with a']

In [ ]:
set_1 = set(errors_tt)
set_2 = set(errors)
print(len(set_1), len(set_2))
print(set_1 - set_2)
print(set_2 - set_1)

In [ ]:
counter_spell_1 = 0
counter_spell_2 = 0
counter_spell_3 = 0
counter_blob_1 = 0
counter_blob_2 = 0
counter_blob_3 = 0

for word in errors_tt:
    if word in spell_set_1: counter_spell_1 += 1 
    if word in spell_set_2: counter_spell_2 += 1
    if word in spell_set_3: counter_spell_3 += 1
    if word in blob_set_1: counter_blob_1 += 1
    if word in blob_set_2: counter_blob_2 += 1
    if word in blob_set_3: counter_blob_3 += 1
print (counter_spell_1, counter_spell_2, counter_spell_3)
print (counter_blob_1, counter_blob_2, counter_blob_3)
    

In [ ]:
for word in errors_tt:
    if word not in blob_set_3: print(word)

In [ ]:
counter_spell_1 = 0
counter_spell_2 = 0
counter_spell_3 = 0
counter_blob_1 = 0
counter_blob_2 = 0
counter_blob_3 = 0

for word in errors:
    if word in spell_set_1: counter_spell_1 += 1 
    if word in spell_set_2: counter_spell_2 += 1
    if word in spell_set_3: counter_spell_3 += 1
    if word in blob_set_1: counter_blob_1 += 1
    if word in blob_set_2: counter_blob_2 += 1
    if word in blob_set_3: counter_blob_3 += 1
print (counter_spell_1, counter_spell_2, counter_spell_3)
print (counter_blob_1, counter_blob_2, counter_blob_3)
    

In [ ]:
print(len(errors_raw), len(errors), len(corrections))

In [ ]:
print(len(corrections))

In [ ]:
for error, correction in zip(errors, corrections):
    if error == correction:
        display_words_in_context(test_words_3, [error], False, 20)

In [ ]:
for word in errors:
    if word not in text:
        display_words_in_context(test_words_3, [word], False, 6)

In [ ]:
display_text_in_context(text, ["never forgive me"])

In [ ]:
print([word in blob_set_3 for word in errors])

In [ ]:
where_is("bethat")

In [ ]:
"be that" in corrections

In [ ]:
"allshot" in both_3

In [ ]:
display_text_in_context(text,["ell"])

In [ ]:
replacements = {error: correction for error, correction in zip (errors, corrections)}

In [ ]:
print(replacements["with-a"])

In [ ]:
print(' '.join(word for word in (blob_only_2 - blob_only_1)))
display_words_in_context(test_words_2, blob_only_2 - blob_only_1, False, 3)

In [ ]:
print(' '.join(word for word in (spell_only_3 - spell_only_2 - spell_only_1)))
display_words_in_context(test_words_3, spell_only_3 - spell_only_2 - spell_only_1, False, 3)

In [ ]:
print(' '.join(word for word in (blob_only_3 - blob_only_2 - blob_only_1)))
display_words_in_context(test_words_3, blob_only_3 - blob_only_2 - blob_only_1, False, 3)

In [ ]:
counter = 0
for word in blob_set_2:
    if word.lower() in blob_set_3:
        print(f"{word} in both")
        print(Word(word).spellcheck())
        if word != word.lower():
            print(Word(word.lower()).spellcheck())
    else:
        print (f"{word} in 2 but not in 3")
        print(Word(word).spellcheck())
    print("\n")    
    counter = counter + 1
    if counter > 15: break    

In [ ]:
errors = "istrue canbe im tj1e overath laval itsrather wer silverss gotto thelikes ifi 0f prise ritz ofthe andthe lto lookingback al1 verypleased het thevery agre bethat seehe ot selfvident sleightsofhand cansee thatsall distinguishedand villagesir docrucially arec farradayscircle witha canonly barnets redding tosit clementss newtmating ill1terest anysuch dh lewiss donttake allhis contemplatin8 startseeing symons owardst imvery simplyaccepting selftraining wellcontented evercourteous donttake allhis lastminute civvy"

In [ ]:
bad_words=["ohnothing", "iarrived", "friendsand", "allshot"]
for word in bad_words:
    print(word in errors_split)

In [ ]:
suggestions_split = suggestions.split(' ')
suggestions_sorted = sorted(suggestions_split)
print(suggestions_sorted)

In [ ]:
errors_sorted = sorted(errors_split)
print (errors_sorted)

In [ ]:
for word in bad_words:
    errors_split.append(word)
print(errors_split)

In [ ]:
print(suggestions)

In [ ]:
errors_split = ['istrue', 'canbe', 'im', 'tj1e', 'overath', 'laval', 'itsrather', 'wer', 'silverss', 'gotto', 'thelikes', 'ifi', '0f', 'prise', 'ritz', 'ofthe', 'andthe', 'lto', 'lookingback', 'al1', 'verypleased', 'het', 'thevery', 'agre', 'bethat', 'seehe', 'ot', 'selfvident', 'sleightsofhand', 'cansee', 'thatsall', 'distinguishedand', 'villagesir', 'docrucially', 'arec', 'farradayscircle', 'witha', 'canonly', 'barnets', 'redding', 'tosit', 'clementss', 'newtmating', 'ill1terest', 'anysuch', 'dh', 'lewiss', 'donttake', 'allhis', 'contemplatin8', 'startseeing', 'symons', 'owardst', 'imvery', 'simplyaccepting', 'selftraining', 'wellcontented', 'evercourteous', 'donttake', 'allhis', 'lastminute', 'civvy', 'ohnothing', 'iarrived', 'friendsand', 'allshot']

In [ ]:


for bad_word in bad_words:
    ind = text.find(bad_word)
    if ind == -1:
        print(f"{bad_word} not found")
    else:
        window = text[ind-50:ind+50]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)
    #test_result = (Word(test_word).spellcheck())
    #print(test_result)


In [ ]:
test_words = text.translate(str.maketrans('', '', string.punctuation)).lower().split()
bad_words=["ohnothing", "iarrived", "friendsand", "allshot"]
bad_words = ["ell", "ting"]
for bad_word in bad_words:
    try:
        ind = test_words.index(bad_word)
        print(f"The index of '{bad_word}' is: {ind}")
        print(' '.join(test_words[ind - 10:ind + 10]))
    except ValueError:
        print(f"'{item_to_find}' not found in the list.")


In [ ]:
print(' '.join(blob_only))

In [ ]:
both = spell_set & blob_set
print(len(both))

In [ ]:
print(' '.join(word for word in both))

In [ ]:
original_errors = "istrue canbe im tj1e overath laval itsrather wer silverss gotto thelikes ifi 0f prise ritz ofthe andthe lto lookingback al1 verypleased het thevery agre bethat seehe ot selfvident sleightsofhand cansee thatsall distinguishedand villagesir docrucially arec farradayscircle witha canonly barnets redding tosit clementss newtmating ill1terest anysuch dh lewiss donttake allhis contemplatin8 startseeing symons owardst imvery simplyaccepting selftraining wellcontented evercourteous donttake allhis lastminute civvy"

In [ ]:
original_errors_split = set(original_errors.split())

In [ ]:
missed = {word for word in original_errors_split if word not in errors}

In [ ]:
print ([(word, word in blob_set_3) for word in missed])

In [ ]:
print ([(word, word in both_3) for word in missed])

In [ ]:
print ([(word, word in spell_set_3) for word in missed])


In [ ]:
for word in errors_split:
    if word not in text:
        if word not in cleaned_suggestions:
            print(word)


In [ ]:
where_is("distinguishedand")

In [ ]:
print([word for word in blob_set_3 if word not in blob_only])

In [ ]:
suggestions = "Overath Laval 1e with-a it'srather Silvers's Farraday'scircle Barnet's Clements's newt-mating Lewis's don'ttake I'mvery self-training well-contented ever-courteous don'ttake last-minute Ritz Redding self-vident sleights-of-hand be-that that'sall d-h Symons"
cleaned_suggestions = suggestions.translate(str.maketrans('', '', string.punctuation)).lower().split()

In [ ]:
for word in suggestions.split(' '):
    if word not in text:
        print(word)

In [ ]:
for word in errors_split:
    if word not in cleaned_suggestions:`
        if word not in test_words:
            print(word)

In [ ]:
" prise " in text